# Using AC-GAN to train Unseen Adversarial Images

In [1]:
import time
import json
import models.resnet_model as resnet_model
from cleverhans import fgm
from models.madry_mnist import MadryModel
from models.aditi_mnist import AditiMNIST
from models.zico_mnist import ZicoMNIST
from utils import *
from adv_utils import *
from models.vgg16 import vgg_16
from models.acwgan_gp import ACWGAN_GP
import argparse
import imageio

Instructions for updating:
non-resource variables are not supported in the long term


In [19]:
def adv_generator(noise=False):
    """adv image generator using ac-gan"""
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    dim_D = 32
    dim_G = 32

    batch_size = 64
    z_dim = 128
    dataset = "mnist"
    checkpoint_dir = "assets/checkpoint"
    result_dir = "assets/result"
    log_dir = "assets/log"
    eps = 0.1

    num_classes = 10
    num_classes = 10
    image_size = 28
    channels = 1

    acgan = ACWGAN_GP(
        sess,
        epoch=10,
        batch_size=batch_size,
        z_dim=z_dim,
        dataset_name=dataset,
        checkpoint_dir=checkpoint_dir,
        result_dir=result_dir,
        log_dir=log_dir,
        dim_D=dim_D,
        dim_G=dim_G
    )

    acgan.build_model()

    adv_list = np.array([])

    for _ in range(10):
        for j in range(10):
            source = j

            source_np = np.asarray([source] * batch_size).astype(np.int32)

            adv_noise = tf.get_variable('adv_noise_' + str(source + '_' + j), shape=(batch_size, image_size, image_size, channels),
                                        dtype=tf.float32, initializer=tf.zeros_initializer)
            adv_z = tf.get_variable('adv_z_' + str(source + '_' + j),
                                    shape=(batch_size, z_dim),
                                    dtype=tf.float32,
                                    initializer=tf.random_normal_initializer)

            # ref_z = tf.get_variable('ref_z_' + str(source),
            #                         shape=(batch_size, z_dim),
            #                         dtype=tf.float32,
            #                         initializer=tf.zeros_initializer)

            adv_images = acgan.generator(adv_z, source_np, reuse=True)

            _, acgan_logits = acgan.discriminator(
                adv_images, update_collection=None, reuse=True)
            acgan_pred = tf.argmax(acgan_logits, axis=1)

            if noise:
                adv_images += eps * tf.nn.tanh(adv_noise)
                if dataset in ('svhn', 'celebA'):
                    adv_images = tf.clip_by_value(
                        adv_images, clip_value_min=-1., clip_value_max=1.0)
                else:
                    adv_images = tf.clip_by_value(
                        adv_images, clip_value_min=0., clip_value_max=1.)

            np.append(adv_list, dict(X=adv_images, y=source_np)))

    return adv_list


In [20]:
"""defense by attack using ac-gans"""
adv_images = adv_generator()

# test accuracy of classifier for adv_images
print(adv_images.shape)

ValueError: Variable discriminator/d_residual_block_shortcut/w already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\Hisku\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\framework\ops.py", line 2101, in __init__
    self._traceback = tf_stack.extract_stack_for_node(self._c_op)
  File "C:\Users\Hisku\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\framework\ops.py", line 3697, in _create_op_internal
    ret = Operation(
  File "C:\Users\Hisku\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 744, in _apply_op_helper
    op = g._create_op_internal(op_type_name, inputs, dtypes=None,
  File "C:\Users\Hisku\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\ops\gen_state_ops.py", line 1750, in variable_v2
    _, _, _op, _outputs = _op_def_library._apply_op_helper(
  File "C